# Hugging Faceで公開されている大規模言語モデルをSageMakerにデプロイ


* 対象モデル
  
  rinna/japanese-gpt-neox-3.6b-instruction-ppo
  
  https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-ppo


### SageMakerライブラリーのインストール

In [7]:
%pip install sagemaker --upgrade


Requirement already up-to-date: sagemaker in /home/vscode/.local/lib/python3.8/site-packages (2.180.0)
Note: you may need to restart the kernel to use updated packages.


### インポート

In [8]:
import sagemaker
import boto3


### IAMロールの取得

In [9]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role_name = 'AmazonSageMaker-ExecutionRole-20230617T201891' # Role name with `AmazonSageMakerFullAccess` policy attached
	role = iam.get_role(RoleName=role_name)['Role']['Arn']


Couldn't call 'get_role' to get Role ARN from role name inspiron14 to get Role path.


### モデル名などのパラメーターを指定

In [10]:
model_id = 'rinna/japanese-gpt-neox-3.6b-instruction-ppo'
instance_type = 'ml.g5.2xlarge'


### SageMakerへデプロイ

In [11]:
from sagemaker.huggingface.model import HuggingFaceModel

# Hub model configuration <https://huggingface.co/models>
hub = {
  'HF_MODEL_ID': model_id, # model_id from hf.co/models
  'HF_TASK':'text-generation'          # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
  env=hub,                            # configuration for loading model from Hub
  role=role,                          # IAM role with permissions to create an endpoint
  transformers_version='4.26',        # Transformers version used
  pytorch_version='1.13',             # PyTorch version used
  py_version='py39',                  # Python version used
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
)


-----------!

### 推論

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)

prompt = [
    {
        "speaker": "ユーザー",
        "text": "コンタクトレンズを慣れるにはどうすればよいですか？"
    },
    {
        "speaker": "システム",
        "text": "これについて具体的に説明していただけますか？何が難しいのでしょうか？"
    },
    {
        "speaker": "ユーザー",
        "text": "目が痛いのです。"
    },
    {
        "speaker": "システム",
        "text": "分かりました、コンタクトレンズをつけると目がかゆくなるということですね。思った以上にレンズを外す必要があるでしょうか？"
    },
    {
        "speaker": "ユーザー",
        "text": "いえ、レンズは外しませんが、目が赤くなるんです。"
    }
]
prompt = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt
]
prompt = "<NL>".join(prompt)
prompt = (
    prompt
    + "<NL>"
    + "システム: "
)


data = {
  'inputs': prompt,
  'parameters': {
    'do_sample': True,
    'max_new_tokens': 128,
    'temperature': 0.7,
    'repetition_penalty': 1.1,
    'pad_token_id': tokenizer.pad_token_id,
    'bos_token_id': tokenizer.bos_token_id,
    'eos_token_id': tokenizer.eos_token_id
  }
}

# request
output = predictor.predict(data)

output


[{'generated_text': 'ユーザー: コンタクトレンズを慣れるにはどうすればよいですか？<NL>システム: これについて具体的に説明していただけますか？何が難しいのでしょうか？<NL>ユーザー: 目が痛いのです。<NL>システム: 分かりました、コンタクトレンズをつけると目がかゆくなるということですね。思った以上にレンズを外す必要があるでしょうか？<NL>ユーザー: いえ、レンズは外しませんが、目が赤くなるんです。<NL>システム: それはコンタクトレンズによる乾燥が原因である可能性が高いです。コンタクトレンズを適切にケアしていない場合や、長時間つけたままにしている場合は、目に負担をかけることがあります。また、レンズの素材によっても乾燥することがあります。これらは、コンタクトレンズメーカーによって異なります。一般的には、コンタクトレンズを正しく使用すれば、目の健康を守ることができます。'}]

### エンドポイントの削除

In [13]:
predictor.delete_endpoint(delete_endpoint_config=False)
predictor.delete_model()
